<a href="https://colab.research.google.com/github/sanaaria/Master-thesis/blob/main/KGE_NFM_%26_NFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
################################
#This script provide a demo of KGE_NFM & NFM, the runtime on one fold mainly takes 40~50 minutes.


In [ ]:
#  !pip uninstall ampligraph

In [ ]:
# pip install rdflib==7.0.0

In [ ]:
# pip install scipy==1.10.0

In [1]:
pip install tensorflow==2.13

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.4
    Uninstalling gast-0.5.4:
      Successfully uninstalled gast-0.5.4
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 1.2.0
    Uninstalling google-auth-oauthlib-1.2.0:
      Successfully uni

In [2]:
import tensorflow as tf
print(tf.__version__)

2.13.0


In [3]:
pip install scikit-learn==0.24.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 15.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [4]:
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder,MinMaxScaler

In [5]:
pip install deepctr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.2/141.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 14.8 MB/s eta 0:00:00
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0


In [6]:
pip install ampligraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.0/204.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.5/575.5 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.9 MB/s 

In [7]:
pip show scikit-learn

Name: scikit-learn
Version: 1.2.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: ampligraph, bigframes, fastai, imbalanced-learn, librosa, mlxtend, qudida, sklearn-pandas, yellowbrick


In [8]:
import ampligraph as ampligraph

In [9]:
print(ampligraph.__version__)

2.0.1


In [10]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import pandas as pd
import ampligraph as ampligraph
from ampligraph.datasets import load_from_csv

In [11]:
# from ampligraph.evaluation import train_test_split_no_unseen,generate_corruptions_for_fit
# # from ampligraph.evaluation import train_test_split_no_unseen
from ampligraph.datasets import load_from_csv


In [12]:
# from ampligraph.evaluation import evaluate_performance
# As of version 1.1.1, Ampligraph removed the 'evaluate_performance' function and instead introduced the 'mrr_score', 'hits_at_k', and 'mean_rank' functions to calculate metrics for evaluating model performance.
# If you are using version 2.0.1, you should be able to use the 'mrr_score', 'hits_at_k', and 'mean_rank' functions to calculate the desired metrics. Here's an example of how you can do this
from ampligraph.evaluation import mrr_score

In [13]:
from ampligraph.evaluation import mrr_score, hits_at_n_score ,mr_score

In [14]:
# from ampligraph.evaluation.common import generate_corruptions
from ampligraph.latent_features.layers.corruption_generation import CorruptionGenerationLayerTrain

In [15]:
# from ampligraph import ComplEx,TransE,DistMult

In [16]:
from ampligraph.latent_features.layers.scoring import ComplEx

In [17]:
from ampligraph.latent_features.layers.scoring import TransE

In [18]:
from ampligraph.latent_features.layers.scoring import DistMult

In [19]:
# from ampligraph.evaluation import evaluate_performance

In [20]:
# from ampligraph.utils import save_model,restore_model
from ampligraph.utils import save_model
from ampligraph.utils import restore_model

In [21]:
import deepctr

In [22]:
from tensorflow.python.keras.models import Model

In [23]:
from tensorflow.python.keras.layers import Dense, Dropout

In [24]:
print(deepctr.__version__)

0.9.3


In [25]:
from keras.layers import LSTM, Lambda, Layer, Dropout

In [26]:
# from tensorflow.python.keras.layers import LSTM, Lambda, Layer, Dropout

In [27]:
# import tf.keras.layers.LSTM

In [28]:
# NFM stands for Neural Factorization Machine, which is a type of neural network designed for recommendation systems.
from deepctr.models import NFM

In [29]:
from deepctr.feature_column import SparseFeat,DenseFeat,get_feature_names

In [30]:
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam,Adagrad,Adamax


In [31]:
from tensorflow import keras

In [32]:



#load data
################################################################

from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [33]:
#data example: yamanishi_08
dt_08 = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/dt_all_08.txt',delimiter='\t',header=None)
# the script reads a csv file using pandas' read_csv function. This function reads the file from the specified path, which in this case is
# /content/drive/MyDrive/data/yamanishi_08/dt_all_08.txt.

dt_08.columns = ['head','relation','tail']
# the columns of the DataFrame dt_08 are set using the columns attribute. The column names are 'head', 'relation', and 'tail'.

In [34]:
#kg
# ##This code is written in Python using the pandas library.
# #The goal of this code is to load two text files,
# which contain Knowledge Graph (KG) data, and concatenate them into a single pandas DataFrame.
# The KG data in these text files consists of triples (head, relation, tail), which are essentially edges in a graph.
# The 'head' is the subject, the 'relation' is the predicate, and the 'tail' is the object.

kg1 = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/kg_data/kegg_kg.txt',delimiter='\t',header=None)
# The pd.read_csv() function reads the specified file and creates a DataFrame. The delimiter='\t' argument tells pandas to use tabs as separators.
# The header=None argument tells pandas that the first row of the file does not contain column names.

kg2 = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/kg_data/yamanishi_uniprot_kg.txt',delimiter='\t',header=None)
#This code is similar to the previous one.

kg = pd.concat([kg1,kg2])
#Concatenate the two DataFrames.
#The pd.concat() function concatenates the input DataFrames into a single DataFrame.

kg.index = range(len(kg))
#Reset the index of the concatenated DataFrame.
#The index attribute of a DataFrame represents the index of the rows.
#This line of code resets the index of the concatenated DataFrame so that it starts from 0 and increments by 1.

kg.columns = ['head','relation','tail']
#Set the column names of the concatenated DataFrame.
#This line of code assigns new column names to the concatenated DataFrame.


#The resulting kg DataFrame contains the combined KG data from both text files.
# The DataFrame has three columns: 'head', 'relation', and 'tail'. The rows represent the triples (head, relation, tail) in the KG.

In [35]:
##This code is a preprocessing step for the neural feature matrix (NFM) used in recommendation systems.
#It performs label encoding and min-max scaling on the categorical data in the input dataframe.
#for nfm input
# The first two lines create two LabelEncoder objects.
# These are used to convert categorical variables into a numerical format that can be understood by machine learning algorithms.
# The LabelEncoder() function is called twice to create two objects, head_le and tail_le.
head_le = LabelEncoder()
tail_le = LabelEncoder()

# The fit() method is called on both objects. This method calculates the necessary parameters to perform the encoding.
head_le.fit(dt_08['head'].values)
tail_le.fit(dt_08['tail'].values)

# The MinMaxScaler is imported from the preprocessing module of the sklearn library. This is used to scale the data.
mms = MinMaxScaler(feature_range=(0,1))


In [36]:

###descriptors preparation

#The drug id and sequence are read from the respective CSV files and stored in a DataFrame called fp_id.
fp_id = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/791drug_struc.csv')['drug_id']

#RThe protein id, protein id sequence, and protein sequence are read from the respective CSV files and stored in a DataFrame called df_proseq.
#The columns of this DataFrame are then renamed to 'pro_id', 'pro_ids', and 'seq'.
df_proseq = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/989proseq.csv')
df_proseq.columns = ['pro_id','pro_ids','seq']

#The pro_id, which represents the unique identifier for each protein, is extracted from the df_proseq DataFrame.
pro_id = df_proseq['pro_id']

# The drug features are read from the morganfp.txt file using the np.loadtxt() function and stored in the drug_feats variable.
drug_feats = np.loadtxt('/content/drive/MyDrive/data/yamanishi_08/morganfp.txt',delimiter=',')

#The protein features are read from the pro_ctd.txt file using the np.loadtxt() function and stored in the pro_feats variable.
pro_feats = np.loadtxt('/content/drive/MyDrive/data/yamanishi_08/pro_ctd.txt',delimiter=',')

# The protein features are then scaled using the MinMaxScaler. The scaled features are stored in the pro_feats_scaled variable.
pro_feats_scaled = mms.fit_transform(pro_feats)

#Next, PCA is applied to reduce the dimensionality of the scaled protein features to 100 components. The reduced features are stored in the pro_feats_scaled2 variable.
pro_feats_scaled2 = PCA(n_components=100).fit_transform(pro_feats_scaled)

#The reduced protein features are then scaled again using the MinMaxScaler. The scaled features are stored in the pro_feats_scaled3 variable.
pro_feats_scaled3 = mms.fit_transform(pro_feats_scaled2)

#Finally, the fp_df and prodes_df DataFrames are created by concatenating the drug id and drug features (represented by drug_feats),
#and the protein id and protein features (represented by pro_feats_scaled3), respectively.
fp_df = pd.concat([fp_id,pd.DataFrame(drug_feats)],axis=1)
prodes_df = pd.concat([pro_id,pd.DataFrame(pro_feats_scaled3)],axis=1)


In [ ]:

#Function
################################################################

# If you want to test other scenarios, just change the data path.
# But it should be noted that the hypermeters in nfm need to be adjusted.
# Typiclly, the l2_reg_dnn & l2_reg_linear = 1e-5 is enough in the warm start.
# For the cold start, the l2_reg_dnn & l2_reg_linear need to be larger, like 1e-3.

data_path = '/content/drive/MyDrive/data/yamanishi_08/data_folds/warm_start_1_10'


######This function is designed to work with 10-fold cross-validation, as it assumes there are 10 different folds for the training and testing sets.
# Therefore, the input i represents the current fold. The function loads the training and testing data for this fold and returns them.
# In addition, the function merges the positive train examples and the knowledge graph into a single dataframe,
# which is used for creating the embeddings. This is why the data dataframe includes only the head, relation, and tail attributes, without the label attribute.
def load_data(i):
    # Read the train_fold csv file. The label is included.
    train = pd.read_csv(data_path+'/train_fold_'+str(i+1)+'.csv')[['head','relation','tail','label']]

    # Select only the positive examples (label == 1) from the train set.
    train_pos = train[train['label']==1]

    # Read the test_fold csv file. The label is included.
    test = pd.read_csv(data_path+'/test_fold_'+str(i+1)+'.csv')[['head','relation','tail','label']]

    # Merge the positive train examples and the knowledge graph into a single dataframe.
    data = pd.concat([train_pos,kg])[['head','relation','tail']]

    # Return the train, train_pos, test, and data dataframes.
    return train,train_pos,test,data



####This Python function, roc_auc, computes the area under the ROC curve (AUC-ROC) for a given binary classification problem.
#It takes two parameters: y (ground truth) and pred (predicted probabilities). The function returns the AUC-ROC score.

#This line defines the function roc_auc that takes two parameters: y and pred.
def roc_auc(y,pred):
  # This line calls the roc_curve function from the metrics module (part of the Scikit-Learn library) with y and pred as parameters.
  #The roc_curve function calculates the ROC curve for the given binary classification problem, returning false positive rate (FPR), true positive rate (TPR), and thresholds.
    fpr, tpr, thresholds = metrics.roc_curve(y, pred)

    #roc_auc = metrics.auc(fpr, tpr): This line calls the auc function from the metrics module with fpr and tpr as parameters.
    #The auc function calculates the area under the ROC curve, which is the AUC-ROC score.
    roc_auc = metrics.auc(fpr, tpr)

    #return roc_auc: This line returns the computed AUC-ROC score.
    return roc_auc



def pr_auc(y, pred):
    precision, recall, thresholds = metrics.precision_recall_curve(y, pred)
    pr_auc = metrics.auc(recall, precision)
    return pr_auc

def get_scaled_embeddings(model,train_triples,test_triples,get_scaled,n_components):
    [train_sub_embeddings,test_sub_embeddings] = [model.get_embeddings(x['head'].values, embedding_type='entity') for x in [train_triples,test_triples]]
    [train_obj_embeddings,test_obj_embeddings] = [model.get_embeddings(x['tail'].values, embedding_type='entity') for x in [train_triples,test_triples]]
    train_feats = np.concatenate([train_sub_embeddings,train_obj_embeddings],axis=1)
    test_feats = np.concatenate([test_sub_embeddings,test_obj_embeddings],axis=1)
    train_dense_features = mms.fit_transform(train_feats)
    test_dense_features = mms.transform(test_feats)
    if get_scaled:
        pca = PCA(n_components=n_components)
        scaled_train_dense_features = pca.fit_transform(train_dense_features)
        scaled_pca_test_dense_features = pca.transform(test_dense_features)
    else:
        scaled_train_dense_features = train_dense_features
        scaled_pca_test_dense_features = test_dense_features
    return scaled_train_dense_features,scaled_pca_test_dense_features


def get_features(data,fp_df,prodes_df,use_pro):
    drug_features = pd.merge(data,fp_df,how='left',left_on='head',right_on='drug_id').iloc[:,4:1029].values
    pro_features = pd.merge(data,prodes_df,how='left',left_on='tail',right_on='pro_id').iloc[:,4:105].values
    if use_pro:
        feature = np.concatenate([drug_features,pro_features],axis=1)
    else:
        feature = drug_features
    return feature


In [ ]:

#'DenseFeat("des",train_des.shape[1]),'des':train_des,' is used for nfm training
def get_nfm_input(re_train_all,re_test_all,train_feats,test_feats,train_des,test_des,embedding_dim,pca_components):
    train_all_feats = np.concatenate([train_feats,train_des],axis=1)
    test_all_feats = np.concatenate([test_feats,test_des],axis=1)
    train_all_feats_scaled = mms.fit_transform(train_all_feats)
    test_all_feats_scaled = mms.transform(test_all_feats)
    feature_columns = [SparseFeat('head',re_train_all['head'].unique().shape[0],embedding_dim=embedding_dim),
                        SparseFeat('tail',re_train_all['tail'].unique().shape[0],embedding_dim=embedding_dim),
                        DenseFeat("feats",train_all_feats_scaled.shape[1]),
                        #DenseFeat("des",train_des.shape[1])
                        ]
    train_model_input = {'head':head_le.transform(re_train_all['head'].values),
                    'tail':tail_le.transform(re_train_all['tail'].values),
                     'feats':train_all_feats_scaled,
                     #'des':train_des
                    }
    test_model_input = {'head':head_le.transform(re_test_all['head'].values),
                    'tail':tail_le.transform(re_test_all['tail'].values),
                    'feats':test_all_feats_scaled,
                    # 'des':test_des
                    }
    return feature_columns,train_model_input,test_model_input


In [ ]:

#the hypermeters(l2_reg_dnn & l2_reg_linear) need to be adjusted in cold start scenarios, like 1e-3
def train_nfm(feature_columns,train_model_input,train_label,test_model_input,y,patience):
    re_model = NFM(feature_columns,feature_columns,task='binary',dnn_hidden_units=(128,128),
                    l2_reg_dnn=1e-5,l2_reg_linear=1e-5,
                    )
    re_model.compile(Adam(1e-3), "binary_crossentropy",
                metrics=[keras.metrics.Precision(name='precision'),], )
    es = EarlyStopping(monitor='loss',patience=patience,min_delta=0.0001,mode='min',restore_best_weights=True)
    history = re_model.fit(train_model_input, train_label,
                        batch_size=20000, epochs=2000,
                        verbose=2,
                        callbacks=[es]
                        )
    pred_y = re_model.predict(test_model_input, batch_size=512)
    roc_nfm = roc_auc(y,pred_y[:,0])
    pr_nfm = pr_auc(y,pred_y[:,0])
    print(roc_nfm)
    print(pr_nfm)
    return roc_nfm,pr_nfm,pred_y[:,0]

def train(i,test_num_neg,train_num_neg,embedding_dim,n_components,use_pro,patience):
    train,train_pos,test,data = load_data(i)
    model = DistMult(batches_count=10000,
        seed=0,
        epochs=50,
        k=400,
        #embedding_model_params={'corrupt_sides':'o'},
        optimizer='adam',
        optimizer_params={'lr':1e-3},
        loss='pairwise', #pairwise
        regularizer='LP',
        regularizer_params={'p':3, 'lambda':1e-5},
        verbose=True)
    model.fit(data.values, early_stopping =True,early_stopping_params=
                {
                    'x_valid': train_pos[['head','relation','tail']].values,       # validation set, here we use training set for validation
                    'criteria':'mrr',         # Uses mrr criteria for early stopping
                    'burn_in': 10,              # early stopping kicks in after 10 epochs
                    'check_interval':2,         # validates every 2th epoch
                    'stop_interval':3,           # stops if 3 successive validation checks are bad.
                    'x_filter': dt_08.values,          # Use filter for filtering out positives
                    'corrupt_side':'o'         # corrupt object (but not at once)
                })


In [ ]:

    #save_model(model, model_name_path = './eg_model/dismult_400_warm_1_10.pkl')
    #model = restore_model(model_name_path='./eg_model/dismult_400_warm_1_10.pkl')
    columns = ['head','relation','tail']
    test_score = model.predict(test[columns])
    test_label = test['label'].values
    #kge performance evaluation
    roc = roc_auc(test_label,test_score)
    pr = pr_auc(test_label,test_score)
    print(roc)
    print(pr)
    #nfm preparation
    re_train_all = train[columns]
    re_test_all = test[columns]
    train_label = train['label']
    train_dense_features,test_dense_features = get_scaled_embeddings(model,re_train_all,re_test_all,False,n_components)
    train_des = get_features(re_train_all,fp_df,prodes_df,use_pro)
    test_des = get_features(re_test_all,fp_df,prodes_df,use_pro)
    feature_columns,train_model_input,test_model_input = get_nfm_input(re_train_all,re_test_all,
                                                                    train_dense_features,test_dense_features,
                                                                    train_des,test_des,
                                                                    embedding_dim,n_components)
    roc_nfm,pr_nfm,pred_y = train_nfm(feature_columns,train_model_input,train_label,test_model_input,test_label,patience)
    return roc,pr,roc_nfm,pr_nfm,re_train_all,train_label,re_test_all,test_label,pred_y



NameError: ignored

In [ ]:

#train and test
#the early stopping parameter in nfm, referring patience, need to be adjusted in cold start scenarios, like 15~20
################################################################
ROC = []
PR = []
ROC_s = []
PR_s = []
for i in range(10):
    print(i)
    roc,pr,roc_s,pr_s,re_train_all,train_label,re_test_all,test_label,pred_y = train(i,10,10,50,200,True,10)
    re_train_all['label'] = train_label
    re_test_all['label'] = test_label
    re_test_all['pred'] = pred_y
    ROC.append(roc)
    PR.append(pr)
    ROC_s.append(roc_s)
    PR_s.append(pr_s)


stable_metrics = pd.DataFrame()
stable_metrics['roc'] = ROC
stable_metrics['pr'] = PR
stable_metrics['roc_s'] = ROC_s
stable_metrics['pr_s'] = PR_s
stable_metrics.describe()


0


TypeError: ignored